In [1]:
%load_ext autoreload
%autoreload 2

import os
import json
import copy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
import ray_results_interpreter as rri
import subprocess
import concurrent.futures
from main_run import MainRun

/user/ml4723/.conda/envs/neural_inventory_control/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/user/ml4723/.conda/envs/neural_inventory_control/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [7]:
mode = "test_on_dev"
setting_name = 'one_warehouse_lost_demand'

models = []

# base_path = "/user/ml4723/Prj/NIC/ray_results/generic_architecture_n_warehouse/vanilla_n_warehouses/run_2025-02-21_18-11-35"
# for subdir in os.listdir(base_path):
#     subdir_path = os.path.join(base_path, subdir)
#     if os.path.isdir(subdir_path):
#         model_path = os.path.join(subdir_path, "model.pt")
#         if os.path.exists(model_path):
#             models.append(model_path)

# base_path = "/user/ml4723/Prj/NIC/ray_results/finals_one_warehouse_n_stores_debug/GNN/run_2025-05-10_15-53-12/run_67955_00001_1_config=one_warehouse_lost_demand,dev_ignore_periods=60,dev_periods=100,different_for_each_sample=True,early_stop_2025-05-10_15-53-12"
# for file in os.listdir(base_path):
#     if file.endswith(".pt"):
#         model_path = os.path.join(base_path, file)
#         models.append(model_path)

base_path = "/user/ml4723/Prj/NIC/ray_results/finals_one_warehouse_n_stores/GNN_selfloop/run_2025-05-15_17-43-23/run_a071c_00007_7_config=one_warehouse_lost_demand,dev_ignore_periods=60,dev_periods=100,different_for_each_sample=True,early_stop_2025-05-15_17-43-24"
models = []
for epoch in range(0, 1001, 10):
    model_path = os.path.join(base_path, f"model_{epoch}.pt")
    models.append(model_path)

def run_main_run(model_path):
    try:
        # Extract hyperparam name from model path
        hyperparam_name = model_path.split('/')[7]
        print(f"Running main_run.py for path {model_path}")
        cmd = [
            "/user/ml4723/.conda/envs/neural_inventory_control/bin/python",
            "main_run.py",
            mode, # test or test_on_dev
            setting_name,
            hyperparam_name,
            model_path,
        ]
        env = {
            **os.environ,
            "MKL_THREADING_LAYER": "GNU",
            "MKL_SERVICE_FORCE_INTEL": "1"
        }
        subprocess.run(cmd, capture_output=True, text=True, check=True, env=env, cwd="/user/ml4723/Prj/NIC/")
    except subprocess.CalledProcessError as e:
        print(f"Error running main_run.py for path {model_path}: {e}")
        print(f"Error output: {e.stderr}")
    except Exception as e:
        print(f"Unexpected error running main_run.py for path {model_path}: {e}")

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for path in models:
        while len(futures) >= 18:
            done, not_done = concurrent.futures.wait(
                futures, 
                return_when=concurrent.futures.FIRST_COMPLETED
            )
            futures = list(not_done)
        futures.append(executor.submit(run_main_run, path))
    concurrent.futures.wait(futures)


Running main_run.py for path /user/ml4723/Prj/NIC/ray_results/finals_one_warehouse_n_stores_debug/GNN/run_2025-05-10_15-53-12/run_67955_00011_11_config=one_warehouse_lost_demand,dev_ignore_periods=60,dev_periods=100,different_for_each_sample=True,early_sto_2025-05-10_15-53-12/model_0.ptRunning main_run.py for path /user/ml4723/Prj/NIC/ray_results/finals_one_warehouse_n_stores_debug/GNN/run_2025-05-10_15-53-12/run_67955_00011_11_config=one_warehouse_lost_demand,dev_ignore_periods=60,dev_periods=100,different_for_each_sample=True,early_sto_2025-05-10_15-53-12/model_10.pt

Running main_run.py for path /user/ml4723/Prj/NIC/ray_results/finals_one_warehouse_n_stores_debug/GNN/run_2025-05-10_15-53-12/run_67955_00011_11_config=one_warehouse_lost_demand,dev_ignore_periods=60,dev_periods=100,different_for_each_sample=True,early_sto_2025-05-10_15-53-12/model_20.pt
Running main_run.py for path /user/ml4723/Prj/NIC/ray_results/finals_one_warehouse_n_stores_debug/GNN/run_2025-05-10_15-53-12/run_6795

In [5]:
models

['/user/ml4723/Prj/NIC/ray_results/generic_architecture_n_warehouse/vanilla_n_warehouses/run_2025-02-21_18-11-35/run_3267f_00003_3_config=n_warehouse_lost_demand,dev_batch_size=32768,dev_n_samples=32768,early_stop_check_epochs=10,learning_rate_2025-02-21_18-11-36/model.pt']

In [12]:
import torch
x = torch.tensor([1.0, float("-inf"), 2.0])
out = torch.softmax(x, dim=0)
print(out)

tensor([0.2689, 0.0000, 0.7311])
